# Clasificare Imagini cu Retele Neuronale Convolutionale

În acest notebook, vom crea două modele CNN pentru a clasifica imagini cu semne de circulație. Vom testa diferiți optimizatori și dimensiuni de kernel și vom analiza performanța modelelor cu diverse metrici. La final, vom folosi SHAP pentru interpretarea deciziilor modelului.

In [ ]:
# Importam librariile necesare pentru construire, antrenare, evaluare si interpretare model CNN
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import os
import cv2
import glob

# ---------------------------------------------------------------------------------
# ENUNT:
# Construim o retea neuronala convolutionala care poate clasifica intre 2 si 10 clase,
# pe o problema la alegerea noastra (ex: emotii, animale, mancare etc).
# Vom construi 2 modele CNN:
#   1) cu 2 layere convolutie + 1 de iesire
#   2) cu 4-6 layere convolutie + 1 de iesire
# Ambele modele vor contine:
# - activare ReLU
# - MaxPooling pentru reducerea dimensiunii spatiale
# - Dropout sau BatchNormalization pentru generalizare mai buna
# - un fully connected layer (Dense)
# - softmax la iesire pentru clasificare multi-clasa
#
# Vom incerca optimizatori SGD si Adam si kernel-uri 3x3 si 5x5.
# Vom evalua performanta cu train/val accuracy, confusion matrix, precision, recall, F1-score,
# si vom afisa curbele de loss si acuratete.
# De asemenea, vom folosi SHAP pentru interpretarea predictiilor modelului.
# ---------------------------------------------------------------------------------


## Încărcarea și preprocesarea datelor

In [ ]:
# ----------------------------
# Functie pentru incarcare si preprocesare date
# ----------------------------

def load_data(data_dir, img_size=(48, 48)):
    """
    Incarca imaginile din folderele cu clase (ex: caini, pisici, trist, fericit etc).
    Redimensioneaza imaginile la dimensiunea img_size.
    Normalizare pixeli intre 0 si 1.
    Returneaza matricea de imagini X, etichetele y (one-hot encoded) si dictionarul label_map.
    """
    X, y = [], []
    labels = os.listdir(data_dir)
    label_map = {label: idx for idx, label in enumerate(labels)}  # fiecare clasa primeste un index numeric
    
    for label in labels:
        files = glob.glob(os.path.join(data_dir, label, '*.png'))  # sau jpg, in functie de dataset
        for file in files:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # incarcare grayscale
            img = cv2.resize(img, img_size)
            img = np.expand_dims(img, axis=-1)  # adauga canalul      
            X.append(img)
            y.append(label_map[label])             
    
    X = np.array(X) / 255.0                     
    y = np.array(y)
    y_cat = to_categorical(y)                     
    return X, y_cat, label_map


## Model 1: CNN simplu (2 layere de convoluție)

In [ ]:
# ----------------------------
# Model 1: CNN simplu cu 2 layere de convolutie + 1 de iesire
# ----------------------------

def build_simple_cnn(input_shape, num_classes, kernel_size=(3,3)):
    """
    Modelul simplu respecta cerinta de 2 layere convolutie + 1 strat de iesire softmax.
    Foloseste activare ReLU, MaxPooling, Dropout si un fully connected layer.
    """
    model = models.Sequential([
        layers.Conv2D(32, kernel_size, activation='relu', input_shape=input_shape),  # strat convolutie 1 cu ReLU
        layers.MaxPooling2D((2, 2)),  # reducere spatiala
        layers.Conv2D(64, kernel_size, activation='relu'),  # strat convolutie 2 cu ReLU
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),  # transformare matrice 2D -> vector 1D pentru dense layer
        layers.Dense(128, activation='relu'),  # fully connected cu ReLU
        layers.Dropout(0.5),  # regularizare Dropout pentru a evita overfitting
        layers.Dense(num_classes, activation='softmax')  # strat de iesire cu softmax pentru clasificare multi-clasa
    ])
    return model


## Model 2: CNN complex (4-6 layere de convoluție)

In [ ]:
# ----------------------------
# Model 2: CNN complex cu 4 layere de convolutie + 1 de iesire
# ----------------------------

def build_complex_cnn(input_shape, num_classes, kernel_size=(3,3)):
    """
    Modelul complex respecta cerinta de 4-6 layere convolutie + 1 strat de iesire.
    Include BatchNormalization pentru stabilizare, Dropout pentru regularizare,
    ReLU, MaxPooling, un fully connected mare si softmax la iesire.
    """
    model = models.Sequential([
        layers.Conv2D(32, kernel_size, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size, activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size, activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, kernel_size, activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


## Antrenare și comparare modele cu Adam și SGD

In [ ]:
# ----------------------------
# Functie de antrenare a modelului cu optimizator specificat
# ----------------------------

def train_model(model, X_train, y_train, X_val, y_val, optimizer, epochs=10):
    """
    Compileaza si antreneaza modelul folosind optimizerul specificat (SGD sau Adam).
    Functia de pierdere folosita este categorical_crossentropy, specifica clasificarii multi-clasa.
    Returneaza istoricul antrenamentului.
    """
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=32)
    return history


## Evaluare performanță

In [ ]:
# ----------------------------
# Evaluare model: raport clasificare si matrice confuzie
# ----------------------------

def evaluate_model(model, X_val, y_val):
    """
    Calculeaza predictiile modelului, afiseaza raportul de clasificare
    (precision, recall, f1-score) si matricea de confuzie (heatmap).
    """
    y_pred = model.predict(X_val)
    y_true = np.argmax(y_val, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    print(classification_report(y_true, y_pred_classes))
    
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


## Plot evoluție Loss și Accuracy

In [ ]:
# ----------------------------
# Afisare grafice pierdere si acuratete
# ----------------------------

def plot_history(history):
    """
    Afiseaza curbele de evolutie a pierderii (loss) si acuratetii (accuracy)
    pe setul de antrenament si validare, pentru analiza performantelor.
    """
    plt.figure(figsize=(12,5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Curba pierderii (Loss)')
    plt.xlabel('Epoca')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Curba acuratetii (Accuracy)')
    plt.xlabel('Epoca')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

## Interpretare cu SHAP

In [ ]:
# ----------------------------
# Interpretare cu SHAP
# ----------------------------

def interpret_with_shap(model, X_train, X_val):
    """
    Folosim SHAP pentru a interpreta deciziile modelului.
    Selectam un subset de imagini ca background si afisam explicatii vizuale
    pentru 2-3 imagini din setul de validare.
    """
    background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]
    explainer = shap.GradientExplainer((model, model.input), background)
    shap_values = explainer.shap_values(X_val[:3])
    shap.image_plot(shap_values, X_val[:3])


## Rulare exemplu

In [ ]:
# ----------------------------
# Exemplu complet de rulare
# ----------------------------

data_dir = 'data'  # folderul cu date: fiecare clasa are propriul subfolder
img_size = (32, 32)  # dimensiune uniforma pentru imagini

# Incarcam si preprocesam datele
X, y, label_map = load_data(data_dir, img_size)
print("Clasele detectate:", label_map)

# Impartim datele in train si validare
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = X_train.shape[1:]
num_classes = y.shape[1]

# Construim modelul simplu cu kernel 3x3 si optimizator Adam
model_simple = build_simple_cnn(input_shape, num_classes, kernel_size=(3,3))
history_simple = train_model(model_simple, X_train, y_train, X_val, y_val, optimizer=Adam(), epochs=10)

# Construim modelul complex cu kernel 5x5 si optimizator SGD
model_complex = build_complex_cnn(input_shape, num_classes, kernel_size=(5,5))
history_complex = train_model(model_complex, X_train, y_train, X_val, y_val, optimizer=SGD(), epochs=10)

# Evaluam si afisam performanta modelului simplu
print("Evaluare model simplu:")
evaluate_model(model_simple, X_val, y_val)
plot_history(history_simple)

# Evaluam si afisam performanta modelului complex
print("Evaluare model complex:")
evaluate_model(model_complex, X_val, y_val)
plot_history(history_complex)

# Interpretare SHAP pentru modelul simplu
interpret_with_shap(model_simple, X_train, X_val)